In [10]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Install requests package for API call
#%pip install requests
import requests
import json

# Frankfurter latest exchange rates, base EUR
url = "https://api.frankfurter.app/latest?from=EUR"

response = requests.get(url)
fx_data = response.json()

print(json.dumps(fx_data, indent=2))
base_currency = fx_data["base"]
date = fx_data["date"]
rates_dict = fx_data["rates"]

records = []

for target_currency, rate in rates_dict.items():
    records.append({
        "base_currency": base_currency,
        "date": date,
        "target_currency": target_currency,
        "rate": float(rate)
    })

print("Number of records:", len(records))
print("First 3 records:")
for r in records[:3]:
    print(r)
df_raw = spark.createDataFrame(records)

df_raw.printSchema()
df_raw.show(10, truncate=False)
from pyspark.sql.functions import col

df_clean = (
    df_raw
    .withColumn("rate", col("rate").cast("double"))
    .withColumn("amount", col("rate"))  # simple duplicated column, just example
)

df_clean.printSchema()
df_clean.show(10, truncate=False)
#df_clean.write.mode("overwrite").format("delta").saveAsTable("fx_rates_latest")
# Store history (append each run)
df_clean.write.mode("append").format("delta").saveAsTable("fx_rates_history")

print("Saved table: fx_rates_history")
spark.sql("""
    SELECT *
    FROM fx_rates_history
    ORDER BY date DESC, target_currency
    LIMIT 10
""").show()

# Quick stats on the history table
spark.sql("""
SELECT 
  COUNT(*) AS num_rows,
  COUNT(DISTINCT date) AS num_dates,
  MIN(date) AS min_date,
  MAX(date) AS max_date,
  MIN(rate) AS min_rate,
  MAX(rate) AS max_rate,
  AVG(rate) AS avg_rate
FROM fx_rates_history
""").show()

# Strongest currencies (lowest rate)
spark.sql("""
SELECT target_currency, rate, date
FROM fx_rates_history
ORDER BY rate ASC
LIMIT 10
""").show()

# Weakest currencies (highest rate)
spark.sql("""
SELECT target_currency, rate, date
FROM fx_rates_history
ORDER BY rate DESC
LIMIT 10
""").show()

spark.sql("SHOW TABLES").show()


#df_clean.toPandas().to_csv("/lakehouse/default/Files/fx_rates_latest.csv", index=False)

#from notebookutils import mssparkutils

#mssparkutils.fs.ls(".")

#mssparkutils.fs.ls("Files")

#mssparkutils.fs.ls("Tables")

#df_clean.toPandas().to_csv("Files/fx_rates_latest.csv", index=False)





StatementMeta(, 597eec1b-a861-464a-894b-2ed9e5e2004c, 12, Finished, Available, Finished)

{
  "amount": 1.0,
  "base": "EUR",
  "date": "2025-12-04",
  "rates": {
    "AUD": 1.7657,
    "BGN": 1.9558,
    "BRL": 6.1892,
    "CAD": 1.6299,
    "CHF": 0.934,
    "CNY": 8.2498,
    "CZK": 24.157,
    "DKK": 7.4686,
    "GBP": 0.8745,
    "HKD": 9.0801,
    "HUF": 382.03,
    "IDR": 19430,
    "ILS": 3.7754,
    "INR": 104.84,
    "ISK": 149.0,
    "JPY": 180.55,
    "KRW": 1716.69,
    "MXN": 21.315,
    "MYR": 4.7971,
    "NOK": 11.782,
    "NZD": 2.0236,
    "PHP": 68.868,
    "PLN": 4.2313,
    "RON": 5.0925,
    "SEK": 10.967,
    "SGD": 1.5115,
    "THB": 37.395,
    "TRY": 49.519,
    "USD": 1.1666,
    "ZAR": 19.8099
  }
}
Number of records: 30
First 3 records:
{'base_currency': 'EUR', 'date': '2025-12-04', 'target_currency': 'AUD', 'rate': 1.7657}
{'base_currency': 'EUR', 'date': '2025-12-04', 'target_currency': 'BGN', 'rate': 1.9558}
{'base_currency': 'EUR', 'date': '2025-12-04', 'target_currency': 'BRL', 'rate': 6.1892}
root
 |-- base_currency: string (nullable = tru